In [10]:
import sys
sys.path.append("..")
import torch
import torch.nn as nn
import torch.nn.functional as F

from models import *
from transformers import GPT2Model, GPT2Config

In [2]:
N, L, d, nh = 7, 5, 6, 2

In [3]:
af_block = AFBlock(MyTfConfig(embed_dim=d, num_heads=nh))
af_block.attn = nn.MultiheadAttention(d, nh, batch_first=True)
af_block.attn.in_proj_weight.data[0:d,:] = torch.eye(d)
af_block.attn.in_proj_weight.data[d:2*d,:] = torch.eye(d)
af_block.attn.in_proj_weight.data[2*d:3*d,:] = torch.eye(d)
af_block.attn.out_proj.weight.data = torch.eye(d)

mha = nn.MultiheadAttention(d, nh, batch_first=True)
mha.in_proj_weight.data[0:d,:] = torch.eye(d)
mha.in_proj_weight.data[d:2*d,:] = torch.eye(d)
mha.in_proj_weight.data[2*d:3*d,:] = torch.eye(d)
mha.out_proj.weight.data = torch.eye(d)

In [4]:
# x = torch.ones(N,L,d)
# x = torch.cat([i * torch.ones(N,1,d) for i in range(1,L+1)], dim=1)
x = torch.cat([1 * torch.ones(N,1,d) for i in range(1,L+1)], dim=1)
bool_mask = torch.triu(torch.ones(L,L), diagonal=1).view(1,L,L).repeat(N*nh,1,1).bool()
float_mask = (1.0 - torch.tril(torch.ones(L,L), diagonal=0)).view(1,L,L).repeat(N*nh,1,1) * -9999

In [5]:
out, wt = mha(x,x,x, attn_mask=bool_mask)
(out.shape, wt.shape, out, wt)

(torch.Size([7, 5, 6]),
 torch.Size([7, 5, 5]),
 tensor([[[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
 

In [6]:
out, wt = mha(x,x,x, attn_mask=float_mask)
(out.shape, wt.shape, out, wt)

(torch.Size([7, 5, 6]),
 torch.Size([7, 5, 5]),
 tensor([[[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]],
 
         [[1., 1., 1., 1., 1., 1.],
 

In [7]:
af_mask = torch.randint(0,2,(N,L))
af_mask[:,0] = 1
af_mask[0,] = torch.ones(L)

out, wt = af_block(x, attention_mask=af_mask)
(out.shape, wt.shape, out, wt)

(torch.Size([7, 5, 6]),
 torch.Size([7, 5, 5]),
 tensor([[[ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402]],
 
         [[ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402]],
 
         [[ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0.4609, -0.5618, -0.0073,  1.9570, -1.2402],
          [ 0.3132, -0

In [8]:
af_mask

tensor([[1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 1, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 1],
        [1, 1, 0, 1, 1],
        [1, 0, 0, 1, 1]])

In [15]:
gpt2 = GPT2Model(GPT2Config())

In [19]:
x = torch.rand(2,10,768)
attn_mask = torch.randint(0,2,(2,10))
out = gpt2(inputs_embeds=x, attention_mask=attn_mask)

In [20]:
out

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[ 1.2171, -0.5624,  1.4131,  ...,  0.9433, -0.7851,  0.1241],
         [ 0.7955, -1.2316, -0.4905,  ..., -0.5310,  1.3153, -1.3016],
         [ 1.0482,  1.0820,  1.9590,  ...,  1.4676, -0.9111, -0.9525],
         ...,
         [ 0.0240, -0.0895,  1.3202,  ..., -0.2257,  0.2048,  1.2907],
         [ 2.0779, -0.1102,  2.1744,  ..., -0.3834, -1.7437,  0.6793],
         [ 0.8970,  0.0741,  1.9502,  ...,  1.8011, -1.1763,  1.5890]],

        [[-0.0715, -0.1641, -1.0609,  ...,  1.1764,  1.9590,  1.2343],
         [ 2.0304, -0.0848,  3.1860,  ...,  0.0866,  0.1336,  0.9036],
         [-0.4762, -1.6202, -0.9255,  ..., -1.1442, -0.8874, -0.4130],
         ...,
         [-0.3769, -0.4215, -0.7306,  ...,  0.4972, -0.6382,  1.9560],
         [ 0.2811, -0.6059, -0.4958,  ...,  0.4321,  0.8852,  2.0337],
         [ 1.6638, -0.7922,  1.5193,  ...,  1.1282,  1.3208,  0.9060]]],
       grad_fn=<ViewBackward0>), past_key_values=((tenso